In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import fpfs
import fitsio
import lensPT as lpt

# Test catalog reading

In [3]:
# prepare the fpfs measurement 
nms=[
    "fpfs_M00",
    "fpfs_M20",
    "fpfs_M22c",
    "fpfs_M22s",
    "fpfs_M40",
    "fpfs_M42c",
    "fpfs_M42s",
]
data=fitsio.read('./data/fpfs-cut32-0000-g1-0000.fits')
ell_fpfs=fpfs.catalog.fpfsM2E(data, const=1., noirev=False)
ell_fpfs_corr=fpfs.catalog.fpfsM2E(data, const=1., noirev=True)
cov_data=lpt.fpfsCov2lptCov(data,nms)

In [4]:
cat=lpt.Catalog(mode_names=['fpfs_M00', 'fpfs_M22c'],data='./data/fpfs-cut32-0000-g1-0000.fits')
cat=lpt.observable.Catalog(mode_names=nms, data='./data/fpfs-cut32-0000-g1-0000.fits', noise_cov=cov_data)

# Test the FPFS ellipticity and shear response

In [5]:
ell1=lpt.fpfs.weighted_e1(Const=1.)
np.testing.assert_array_almost_equal(ell1.evaluate(cat), ell_fpfs['fpfs_e1'])

ell2=lpt.fpfs.weighted_e2(Const=1.)
np.testing.assert_array_almost_equal(ell2.evaluate(cat), ell_fpfs['fpfs_e2'])

In [6]:
# compare the result with the original FPFS code
de1_dg=lpt.g1Perturb1(ell1)
np.testing.assert_array_almost_equal(de1_dg.evaluate(cat), ell_fpfs['fpfs_R1E'])
de2_dg=lpt.g2Perturb1(ell2)
np.testing.assert_array_almost_equal(de2_dg.evaluate(cat), ell_fpfs['fpfs_R2E'])

# Test the noise bias correction

In [7]:
noiseE1=lpt.noisePerturb2(ell1)
de1_noise = noiseE1.evaluate(cat)
de1_noise_fpfs=ell_fpfs['fpfs_e1'] - ell_fpfs_corr['fpfs_e1']
np.testing.assert_array_almost_equal(de1_noise, de1_noise_fpfs)

In [8]:
noiseE2=lpt.noisePerturb2(ell2)
de2_noise = noiseE2.evaluate(cat)
de2_noise_fpfs=ell_fpfs['fpfs_e2'] - ell_fpfs_corr['fpfs_e2']
np.testing.assert_array_almost_equal(de2_noise, de2_noise_fpfs)

In [9]:
noiseR1=lpt.noisePerturb2(de1_dg)
noiseR2=lpt.noisePerturb2(de2_dg)

In [14]:
np.testing.assert_array_almost_equal(noiseR1.evaluate(cat), ell_fpfs['fpfs_R1E'] - ell_fpfs_corr['fpfs_R1E'])
np.testing.assert_array_almost_equal(noiseR2.evaluate(cat), ell_fpfs['fpfs_R2E'] - ell_fpfs_corr['fpfs_R2E'])